In [1]:
import os
import json

import openai
from openai import OpenAI
from dotenv import load_dotenv
import tiktoken


import sys
sys.path.append("/Users/andrew/ExpertAI/MSEQA_for_NER/src/MSEQA_4_NER")
from data_handlers import data_handler_pileNER

In [2]:
# Load API key from .env
load_dotenv('./.env')

True

In [3]:
print(os.environ.get('OPENAI_API_KEY') is not None)

True


In [4]:
client = OpenAI()

In [5]:
gpt_model = "gpt-3.5-turbo-1106"

In [6]:
with open("../data_handlers/questions/ne_types_list.json", 'r') as file:
        ne_types_list = json.load(file)

print("NE types which number of occurrences is > 100:")
print(len(ne_types_list))
print(ne_types_list)

NE types which number of occurrences is > 100:
455
['person', 'organization', 'location', 'concept', 'product', 'variable', 'date', 'medical condition', 'object', 'technology', 'chemical', 'software', 'event', 'number', 'disease', 'attribute', 'protein', 'group', 'material', 'measurement', 'other', 'function', 'entity type', 'nationality', 'country', 'class', 'process', 'title', 'animal', 'component', 'condition', 'substance', 'food', 'city', 'activity', 'type', 'company', 'time', 'method', 'property', 'organism', 'drug', 'medical procedure', 'treatment', 'profession', 'cell type', 'anatomical structure', 'job title', 'data', 'quantity', 'sports team', 'biological process', 'data type', 'programming language', 'occupation', 'file', 'body part', 'medical treatment', 'language', 'chemical compound', 'entity', 'gene', 'state', 'law', 'action', 'website', 'library', 'facility', 'publication', 'tool', 'field of study', 'compound', 'document', 'organ', 'abbreviation', 'character', 'brand', '

In [7]:
with open("../data_handlers/questions/sentences_per_ne_type.json", 'r') as file:
    sentences_per_ne_type = json.load(file)

In [8]:
ne_type = 'location'
ex_sentences = sentences_per_ne_type[ne_type]
prompt = data_handler_pileNER.generate_prompt(ne_type, ex_sentences)

print(prompt)

Given a Named Entity and some exemplary sentences in input, provide, concisely, both a definition and warnings about what should not be labelled for this Named Entity. Here is the input: 'Named Entity': location, 'exemplary sentences': "Hastings" and "England" are "location" in the sentence "Turret Clockmakers Thwaites & Reed Ltd Hastings England"."; "New York" is "location" in the sentence ""Patti Smith" came to New York at the age of nineteen, determined to become someone."; "Time Out London" is "location" in the sentence "Talking with Time Out London last week, though, she reportedly revealed that's not the case."; Output in JSON format: {{"Definition": "", "Do not label": ""}}. Limit response to 50 tokens.


In [18]:
system_message = "You are a helpful NER data annotator designed to output JSON in at most 50 tokens."
user_message = prompt

In [19]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-1106"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [20]:
messages_to_send = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

In [21]:
n_input_tokens = num_tokens_from_messages(messages_to_send, gpt_model)
print(n_input_tokens)

200


In [22]:
completion = client.chat.completions.create(
    messages=messages_to_send,
    model=gpt_model,
    max_tokens=75,
    response_format={ "type": "json_object" }
)

In [14]:
print(completion)
print(completion.usage)

ChatCompletion(id='chatcmpl-8VGjJdtskCdqKvLo0X4Ll2KigiUMx', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{\n  "Definition": "An element refers to a substance made up of atoms with the same number of protons, identified by its atomic number.",\n  "Do not label": "Non-elemental concepts, compounds, mixtures, and non-specific entity references like \'quantity of\' or \'amount of\'."\n}', role='assistant', function_call=None, tool_calls=None))], created=1702463169, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_f3efa6edfc', usage=CompletionUsage(completion_tokens=64, prompt_tokens=220, total_tokens=284))
CompletionUsage(completion_tokens=64, prompt_tokens=220, total_tokens=284)


In [15]:
choice = completion.choices[0]
print(choice)

Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{\n  "Definition": "An element refers to a substance made up of atoms with the same number of protons, identified by its atomic number.",\n  "Do not label": "Non-elemental concepts, compounds, mixtures, and non-specific entity references like \'quantity of\' or \'amount of\'."\n}', role='assistant', function_call=None, tool_calls=None))


In [16]:
finish_reason = choice.finish_reason
print(f"finish_reason: {finish_reason}")

message = choice.message
print(message.role)
print(message.content)

finish_reason: stop
assistant
{
  "Definition": "An element refers to a substance made up of atoms with the same number of protons, identified by its atomic number.",
  "Do not label": "Non-elemental concepts, compounds, mixtures, and non-specific entity references like 'quantity of' or 'amount of'."
}


In [24]:
choice = completion.choices[0]
print(completion.usage)
print(choice)
finish_reason = choice.finish_reason
print(f"finish_reason: {finish_reason}")

message = choice.message
print(message.role)
print(message.content)

CompletionUsage(completion_tokens=59, prompt_tokens=200, total_tokens=259)
Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{\n  "Definition": "A location refers to a specific place or position on the Earth\'s surface, often with a name that identifies it uniquely.",\n  "Do not label": "Generic terms like \'here\', \'there\', or \'somewhere\' should not be labeled as locations."\n}', role='assistant', function_call=None, tool_calls=None))
finish_reason: stop
assistant
{
  "Definition": "A location refers to a specific place or position on the Earth's surface, often with a name that identifies it uniquely.",
  "Do not label": "Generic terms like 'here', 'there', or 'somewhere' should not be labeled as locations."
}
